In [ ]:
# python model_main_tf2.py --pipeline_config_path=D:\Study\TeamProject_2\faster_rcnn_resnet50_v1_640x640_coco17_tpu-8.config --model_dir=D:\Study\TeamProject_2\model --alsologtostderr

In [15]:
import tensorflow as tf
from object_detection.builders import model_builder

# 설정 파일 경로
config_path = "D:\Study\TeamProject_2\faster_rcnn_resnet50_v1_640x640_coco17_tpu-8.config"

# 모델 빌더 생성
model_builder = model_builder.ModelBuilder()

# 모델 구조 정의
model = model_builder.build(model_config_path=config_path, is_training=False)

# 모델 구조 확인
print(model)

# 체크포인트 파일 경로
checkpoint_path = 'D:\Study\TeamProject_2\ckpt-26.data-00000-of-00001'

# 체크포인트 로더
checkpoint = tf.train.Checkpoint(model=model)
checkpoint.restore(checkpoint_path)

# 학습이 완료된 모델 객체 생성
trained_model = model

TypeError: Descriptors cannot not be created directly.
If this call came from a _pb2.py file, your generated code is out of date and must be regenerated with protoc >= 3.19.0.
If you cannot immediately regenerate your protos, some other possible workarounds are:
 1. Downgrade the protobuf package to 3.20.x or lower.
 2. Set PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python (but this will use pure-Python parsing and will be much slower).

More information: https://developers.google.com/protocol-buffers/docs/news/2022-05-06#python-updates

In [ ]:
import tensorflow as tf

# 학습이 완료된 모델 객체
trained_model = ...

# 모델 저장 경로
save_path = 'D:\Study\TeamProject_2'

# 모델 저장
tf.saved_model.save(trained_model, save_path)

In [3]:
import cv2
import numpy as np
import tensorflow as tf
from object_detection.utils import visualization_utils as vis_util

# 이미지 로드
image_path = 'D:\Study\TeamProject_2\dataset_filtered\C000002_003_0009_C_D_F_0.jpg'
image = cv2.imread(image_path)
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# 이미지 전처리
input_image = np.expand_dims(image_rgb, axis=0)
input_tensor = tf.convert_to_tensor(input_image, dtype=tf.float32)
input_tensor = tf.image.resize(input_tensor, (320, 320))
input_tensor /= 255.0

# 모델 추론
detections = model.predict(input_tensor)

# 시각화
category_index = {1: {'id': 1, 'name': 'object'}}
vis_util.visualize_boxes_and_labels_on_image_array(
    image_rgb,
    detections['detection_boxes'][0].numpy(),
    detections['detection_classes'][0].numpy().astype(np.int32),
    detections['detection_scores'][0].numpy(),
    category_index,
    instance_masks=detections['detection_masks'][0].numpy(),
    use_normalized_coordinates=True,
    line_thickness=8,
    min_score_thresh=0.5
)

# 시각화 결과 출력
cv2.imshow('Object Detection', image_rgb)
cv2.waitKey(0)
cv2.destroyAllWindows()

NameError: name 'model' is not defined

In [ ]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

# 파이프라인 설정 파일 경로
pipeline_config_path = 'D:\Study\TeamProject_2\faster_rcnn_resnet101_v1_640x640_coco17_tpu-8.config'

# 파이프라인 설정 파일 읽기
configs = config_util.get_configs_from_pipeline_file(pipeline_config_path)

# 모델 설정에 따라 모델 빌드
detection_model = model_builder.build(model_config=configs['model'], is_training=True)

# 라벨맵 파일 경로
label_map_path = '/path/to/your/label_map.pbtxt'

# 라벨맵 로드
label_map = label_map_util.load_labelmap(label_map_path)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=90)
category_index = label_map_util.create_category_index(categories)

# 훈련 루프 설정
train_steps = 200000
strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
  # 훈련을 위한 파이프라인 설정
  pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
  with tf.io.gfile.GFile(pipeline_config_path, "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config)

  # 모델 설정
  model_config = pipeline_config.model
  detection_model = model_builder.build(model_config=model_config, is_training=True)

  # ... (생략)
  # 여기서 훈련에 필요한 세부 설정을 수행할 수 있습니다. 

# 훈련 루프
for step in range(1, train_steps+1):
  # ... (생략)
  # 여기서 훈련에 필요한 입력 데이터를 준비하고, 
  # detection_model.train() 메소드를 호출하여 모델을 훈련시킵니다